In [1]:
import random

from numpy.random import multivariate_normal
from numpy.random import uniform

In [2]:
def init_toy_data(num_samples,num_features, num_classes, seed=3):
    # num_samples: number of samples *per class*
    # num_features: number of features (excluding bias)
    # num_classes: number of class labels
    # seed: random seed
    np.random.seed(seed)
    X=np.zeros((num_samples*num_classes, num_features))
    y=np.zeros(num_samples*num_classes)
    for c in range(num_classes):
        # initialize multivariate normal distribution for this class:
        # choose a mean for each feature
        means = uniform(low=-10, high=10, size=num_features)
        # choose a variance for each feature
        var = uniform(low=1.0, high=5, size=num_features)
        # for simplicity, all features are uncorrelated (covariance between any two features is 0)
        cov = var * np.eye(num_features)
        # draw samples from normal distribution
        X[c*num_samples:c*num_samples+num_samples,:] = multivariate_normal(means, cov, size=num_samples)
        # set label
        y[c*num_samples:c*num_samples+num_samples] = c
    return X,y


In [13]:
import numpy as np

class Network(object):

    
    # initialize network
    def __init__(self, sizes, activations):
        if len(sizes) -1 != len(activations):
            print("number of activations must be equal to number of layers - 1")
        else:
            self.num_layers = len(sizes)
            self.activations = activations
            self.sizes = sizes
            self.biases = [np.random.randn(y) for y in sizes[1:]]
            self.weights = [np.random.randn(y, x) for x, y in zip(sizes[1:], sizes[:-1])]
            self.activation_funcs = {
                "relu" : self.relu,
                "softmax" : self.softmax,
                "sigmoid" : self.sigmoid,
                "identity": self.identity
            }
     
    # activation functions and derivatives
    def sigmoid(self, z):
        return 1.0/(1.0+np.exp(-z))
    
    def sigmoid_prime(self, z):
        return self.sigmoid(z)*(1-self.sigmoid(z))
    
    #----------------------------------------
    
    def relu(self, z):
        return np.maximum(0, z)
    
    def relu_prime(self, z):
        z[z > 0] = 1
        z[z <= 0] = 0
        return z
    
    #----------------------------------------
    
    def softmax(self, x):
        e_x = np.exp(x - np.max(x))
        return e_x / e_x.sum(axis=0) 
    
    #----------------------------------------
    
    def identity(self, x):
        return x

    
    # feed forward
    def feedforward(self, a):
        for b, w, activation in zip(self.biases, self.weights, self.activations):
            activation = self.activation_funcs[activation]
            a = activation(np.dot(a, w)+b)
        return a

In [5]:
X,y= init_toy_data(2,4,3)


In [6]:

#res = []
#for i in preds:
#    res.append(np.where(i == np.amax(i))[0][0])
#print(res, y)

test_data = []
for a, s in zip(X, y):
    test_data.append([a, s])

In [25]:
net = Network([4, 10, 3], ["sigmoid", "softmax"])
net.feedforward(X)

array([[0.01418707, 0.30663032, 0.02499046],
       [0.01117907, 0.550413  , 0.01685667],
       [0.28230429, 0.00303681, 0.06304545],
       [0.36845513, 0.00386532, 0.06154979],
       [0.14444801, 0.09434476, 0.52855822],
       [0.17942643, 0.0417098 , 0.30499941]])